<a href="https://colab.research.google.com/github/kotosham/sleep-stages/blob/main/phase_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/3.csv')
data3 = data3.drop(columns=['Time,s'], axis=1)
df3 = pd.DataFrame(data3)

In [4]:
dataframes = []

for i in range(1, 8):
  if i == 3:
    continue
  url = f'https://raw.githubusercontent.com/kotosham/sleep-stages/refs/heads/main/{i}.csv'
  data = pd.read_csv(url)
  data = data.drop(columns=['Time,s'], axis=1)
  dataframes.append(pd.DataFrame(data))

dataframes.insert(2, df3)
df1 = dataframes[0]
df2 = dataframes[1]
df4 = dataframes[3]
df5 = dataframes[4]
df6 = dataframes[5]
df7 = dataframes[6]

<ipython-input-4-6d7dece21c35>:7: DtypeWarning: Columns (5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)
<ipython-input-4-6d7dece21c35>:7: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)


In [5]:
df4['1 Anadh'] = df4['1 Anadh'].replace('#DIV/0!', '0')
df4['2 Anadh'] = df4['2 Anadh'].replace('#DIV/0!', '0')
df4['1 Anadh'] = df4['1 Anadh'].astype(float)
df4['2 Anadh'] = df4['2 Anadh'].astype(float)

In [6]:
df1

,1 LDF,1 T,1 A365,1 A460,1 Anadh,2 LDF,2 T,2 A365,2 A460,2 Anadh,Sleep stages
0,18.1,0.00,0.0,0.0,0.000000,21.1,0.0,0.0,0.0,0.0,W
1,18.1,0.00,0.0,0.0,0.000000,21.1,0.0,0.0,0.0,0.0,W
2,18.1,0.00,0.0,0.0,0.000000,21.1,0.0,0.0,0.0,0.0,W
3,18.1,0.00,0.0,0.0,0.000000,21.1,0.0,0.0,0.0,0.0,W
4,19.3,33.72,104.0,65.0,0.625000,20.0,0.0,0.0,0.0,0.0,W
...,...,...,...,...,...,...,...,...,...,...,...
191696,22.0,36.83,128.0,42.0,0.328125,NaN,NaN,NaN,NaN,NaN,N2
191697,21.7,36.83,128.0,42.0,0.328125,NaN,NaN,NaN,NaN,NaN,N2
191698,21.7,36.83,128.0,42.0,0.328125,NaN,NaN,NaN,NaN,NaN,N2
191699,21.7,36.83,128.0,42.0,0.328125,NaN,NaN,NaN,NaN,NaN,N2


In [7]:
dataframes = [df1, df2, df3, df4, df5, df6, df7]
label_encoder = LabelEncoder()

for i in range(len(dataframes)):
    dataframes[i] = dataframes[i].dropna(subset=['Sleep stages'])
    dataframes[i] = dataframes[i].fillna(0)
    dataframes[i]['Sleep stages'] = label_encoder.fit_transform(dataframes[i]['Sleep stages'])

df1 = dataframes[0]
df2 = dataframes[1]
df3 = dataframes[2]
df4 = dataframes[3]
df5 = dataframes[4]
df6 = dataframes[5]
df7 = dataframes[6]

In [8]:
df4

,1 LDF,1 T,1 A365,1 A460,1 Anadh,2 LDF,2 T,2 A365,2 A460,2 Anadh,Sleep stages
0,24.7,36.46,131.0,64.0,0.48855,44.1,36.0,125.0,73.0,0.584,3
1,24.7,36.46,131.0,64.0,0.48855,44.1,36.0,125.0,73.0,0.584,3
2,24.7,36.46,131.0,64.0,0.48855,44.1,36.0,125.0,73.0,0.584,3
3,24.7,36.46,131.0,64.0,0.48855,44.1,36.0,125.0,73.0,0.584,3
4,24.1,36.46,131.0,64.0,0.48855,43.0,36.0,125.0,73.0,0.584,3
...,...,...,...,...,...,...,...,...,...,...,...
228142,0.0,0.00,0.0,0.0,0.00000,44.4,38.8,106.0,58.0,0.000,2
228143,0.0,0.00,0.0,0.0,0.00000,41.9,38.8,106.0,58.0,0.000,2
228144,0.0,0.00,0.0,0.0,0.00000,41.9,38.8,106.0,58.0,0.000,2
228145,0.0,0.00,0.0,0.0,0.00000,41.9,38.8,106.0,58.0,0.000,2


In [24]:
phase_datasets = {0: [], 1: [], 2: [], 3: []}

for df in dataframes:
    for phase in range(4):
        phase_data = df[df['Sleep stages'] == phase]
        phase_datasets[phase].append(phase_data)

final_datasets = {}

for phase in range(4):
    final_datasets[phase] = pd.concat(phase_datasets[phase], ignore_index=True)
    final_datasets[phase] = final_datasets[phase].drop(columns=[col for col in final_datasets[phase].columns if 'Unnamed' in col], errors='ignore')

file_names = [f'sleep_phase_{phase}.csv' for phase in range(4)]
save_directory = '/content/drive/My Drive/Colab Notebooks/data/'

for phase in range(4):
    final_datasets[phase].to_csv(f'{save_directory}{file_names[phase]}', index=False)

# Теперь у вас есть четыре DataFrame: final_datasets[0], final_datasets[1], final_datasets[2], final_datasets[3]

In [28]:
final_datasets[3]

,1 LDF,1 T,1 A365,1 A460,1 Anadh,2 LDF,2 T,2 A365,2 A460,2 Anadh,Sleep stages
0,18.1,0.00,0.0,0.0,0.000,21.1,0.0,0.0,0.0,0.0,3
1,18.1,0.00,0.0,0.0,0.000,21.1,0.0,0.0,0.0,0.0,3
2,18.1,0.00,0.0,0.0,0.000,21.1,0.0,0.0,0.0,0.0,3
3,18.1,0.00,0.0,0.0,0.000,21.1,0.0,0.0,0.0,0.0,3
4,19.3,33.72,104.0,65.0,0.625,20.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...
286309,24.6,37.71,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,3
286310,24.6,37.71,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,3
286311,24.6,37.71,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,3
286312,24.6,37.71,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,3
